In [1]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [2]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+14]
        df = pd.read_csv(csv_file, header=[header], on_bad_lines='skip')
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, error_method='Cramer', error_label=' Error (SD)', output='both'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise KeyError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    down_amp = df[down].astype(float).sum(axis=1)
    if error_method.lower() == 'cramer':
        up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
        down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    elif error_method.lower()=='narrow':
        up_err = df[up_err_name].astype(float).max(axis=1) 
        down_err = df[down_err_name].astype(float).max(axis=1) 
    else:
        raise KeyError('Please choose "Cramer" or "Narrow" for the error_method')
    
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label =f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD) {error_method}'
    
    if output=='both':
        return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})
    elif output=='ratio':
        return pd.DataFrame({col_name: ratio})      
    elif output=='error':
        return pd.DataFrame({col_error_name: ratio_err})   
    else:
        raise KeyError('Please choose "ratio", "error" or "both" (default) as output')

In [3]:
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/Ellie')
# data_folder = Path('/Users/papo/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')
data_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')

azi_data = Path('/Users/patxi/Desktop/AZI_MRS')
Ray_phd = Path('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs')
data_folder = Ray_phd

# basis_folder = Path('/Users/papo/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM')
basis_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "csi" in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_both.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    # print(csv)
    if not csv.is_file():
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        command = f'tarquin --rows 3 --cols 2 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        print(f'{command}\n') 
        os.system(command)


  0%|          | 0/5 [00:00<?, ?it/s]

tarquin --rows 3 --cols 2 --input /Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP684/C3839474_CSI_PB_auto_TR2S_SENSE_5_2_raw_act.SDAT --output_csv /Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP684/C3839474_CSI_PB_auto_TR2S_SENSE_5_2_raw_act_both.csv --input_w /Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP684/C3839474_CSI_PB_auto_TR2S_SENSE_5_2_raw_ref.SDAT --basis_csv /Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM


2022-Nov-24 09:44:58 INFO   : TARQUIN 4.3.10 Started
2022-Nov-24 09:44:58 INFO   : Loading data file : /Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP684/C3839474_CSI_PB_auto_TR2S_SENSE_5_2_raw_act.SDAT
2022-Nov-24 09:44:58 INFO   : Loading data file : /Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP684/C3839474_CSI_PB_auto_TR2S_SENSE_5_2_raw_act.SDAT
2022-Nov-24 09:44:58 INFO   : nStart = 20
2022-Nov-24 09:44:58 INFO   : nEnd   = 1024
TARQUIN Version 4.3.10

**** TARQUIN is a research t

.............


2022-Nov-24 09:45:04 INFO   : done.
2022-Nov-24 09:45:04 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM/GPC.csv'
2022-Nov-24 09:45:04 INFO   : done.
2022-Nov-24 09:45:04 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine_no_MM/Glc.csv'

.........................

...


2022-Nov-24 09:46:01 INFO   : done.
2022-Nov-24 09:46:01 DEBUG  : Final Beta: 57.52
2022-Nov-24 09:46:01 DEBUG  : Final Phi0: 0.87
2022-Nov-24 09:46:01 DEBUG  : Final Phi1: -0.00
2022-Nov-24 09:46:01 INFO   : FWHM right ppm = 3.199142
2022-Nov-24 09:46:01 INFO   : FWHM left ppm = 3.258361
2022-Nov-24 09:46:01 INFO   : Metabolite FWHM (ppm) = 0.059219
2022-Nov-24 09:46:01 INFO   : Spec noise            = 0.010667
2022-Nov-24 09:46:01 INFO   : sdev noise = 0.589577
2022-Nov-24 09:46:01 INFO   : SNR residual = 2.776454
2022-Nov-24 09:46:01 INFO   : SNR max      = 3.391050
2022-Nov-24 09:46:01 INFO   : Ymax       = 3.273864
2022-Nov-24 09:46:01 INFO   : Ymax metab = 3.067931
2022-Nov-24 09:46:01 INFO   : Fit quality = 1.221360
2022-Nov-24 09:46:01 INFO   : Baseline dev = 0.045791
2022-Nov-24 09:46:01 INFO   : Ymax         = 3.273864
2022-Nov-24 09:46:01 INFO   : BSL         = -0.022815
2022-Nov-24 09:46:01 INFO   : Computing CRLBs.
2022-Nov-24 09:46:01 DEBUG  : Sigma: 1.345189e-04
2022-No

.....................................................................................................................................


2022-Nov-24 09:46:01 INFO   : done.
2022-Nov-24 09:46:01 DEBUG  : Final Beta: 49.25
2022-Nov-24 09:46:01 DEBUG  : Final Phi0: 0.06
2022-Nov-24 09:46:01 DEBUG  : Final Phi1: -0.00
2022-Nov-24 09:46:01 INFO   : FWHM right ppm = 3.195321
2022-Nov-24 09:46:01 INFO   : FWHM left ppm = 3.243078
2022-Nov-24 09:46:01 INFO   : Metabolite FWHM (ppm) = 0.047757
2022-Nov-24 09:46:01 INFO   : Spec noise            = 0.010937
2022-Nov-24 09:46:01 INFO   : sdev noise = 0.621049
2022-Nov-24 09:46:01 INFO   : SNR residual = 5.365126
2022-Nov-24 09:46:01 INFO   : SNR max      = 6.732255
2022-Nov-24 09:46:01 INFO   : Ymax       = 6.664016
2022-Nov-24 09:46:01 INFO   : Ymax metab = 6.664016
2022-Nov-24 09:46:01 INFO   : Fit quality = 1.254818
2022-Nov-24 09:46:01 INFO   : Baseline dev = 0.020512
2022-Nov-24 09:46:01 INFO   : Ymax         = 6.664016
2022-Nov-24 09:46:01 INFO   : BSL         = -0.020261
2022-Nov-24 09:46:01 INFO   : Computing CRLBs.
2022-Nov-24 09:46:01 DEBUG  : Sigma: 1.546664e-04
2022-No

100%|██████████| 5/5 [01:04<00:00, 12.90s/it].............................................................................................................


2022-Nov-24 09:46:02 INFO   : done.
2022-Nov-24 09:46:02 DEBUG  : Final Beta: 94.44
2022-Nov-24 09:46:02 DEBUG  : Final Phi0: -0.61
2022-Nov-24 09:46:02 DEBUG  : Final Phi1: 0.00
2022-Nov-24 09:46:02 INFO   : FWHM right ppm = 3.184815
2022-Nov-24 09:46:02 INFO   : FWHM left ppm = 3.236392
2022-Nov-24 09:46:02 INFO   : Metabolite FWHM (ppm) = 0.051578
2022-Nov-24 09:46:02 INFO   : Spec noise            = 0.011234
2022-Nov-24 09:46:02 INFO   : sdev noise = 0.626477
2022-Nov-24 09:46:02 INFO   : SNR residual = 5.029739
2022-Nov-24 09:46:02 INFO   : SNR max      = 6.197864
2022-Nov-24 09:46:02 INFO   : Ymax       = 6.302030
2022-Nov-24 09:46:02 INFO   : Ymax metab = 6.302030
2022-Nov-24 09:46:02 INFO   : Fit quality = 1.232244
2022-Nov-24 09:46:02 INFO   : Baseline dev = 0.016549
2022-Nov-24 09:46:02 INFO   : Ymax         = 6.302030
2022-Nov-24 09:46:02 INFO   : BSL         = -0.004391
2022-Nov-24 09:46:02 INFO   : Computing CRLBs.
2022-Nov-24 09:46:02 DEBUG  : Sigma: 1.527222e-04
2022-No

In [4]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act_both.csv" in f.name.lower() and "csi" in f.name.lower())]



rows_to_process = {'BGT': 5, #(BGT = Basal Ganglia and Thalamic, ie deep GM)
                   'WM': 3,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        data_list.append([*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+8),*pull_from_csv(file,header=16,row=row_num+1)])
    my_columns = list(pd.read_csv(file, header=[1], on_bad_lines='skip'))
    my_errors = [item+' Error (SD)' for item in my_columns]
    my_diagnostics = pull_from_csv(file,row=0, header=16)    
    df = pd.DataFrame(data_list, columns=['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ])
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    df_L2N_narrow_error = met_ratio(df,['Lac', 'Threonine'], 'TNAA', error_method='Narrow', output='error')
    df_N2C = met_ratio(df,'TNAA', 'TCho')
    df_C2C = met_ratio(df,'TCho', 'Cr')
    df_N2Cr = met_ratio(df,'TNAA', 'Cr')


    # df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    df_ratio = pd.concat((df_L2N,df_L2N_narrow_error, df_N2C, df_C2C, df_N2Cr, df), axis=1)

    # df_ratio.set_index('LWP').to_excel(f'{label}_RATIOS_INSPIRE.xlsx')
    # df_ratio.set_index('LWP').to_excel(f'{label}_RATIOS_AZI.xlsx')
    df_ratio.set_index('LWP').to_excel(f'{label}_RAY_PLOTS.xlsx')



In [5]:
dfbb = met_ratio(df,'TNAA', 'TCho', error_method='Narrow', output='error')
dfbb


,TNAA/TCho Error (SD) Narrow
0,0.686243
1,0.501901
2,0.695159
3,NaN
4,0.692715


In [6]:
data_list

[['LWPs/LWP684/C3',
  '3',
  '5',
  '1',
  '4.269192e+00',
  '0.000000e+00',
  '0.000000e+00',
  '8.886865e+00',
  '0.000000e+00',
  '1.565578e+00',
  '0.000000e+00',
  '1.268189e+01',
  '1.419094e+01',
  '2.125582e+00',
  '4.171831e+00',
  '4.070623e+00',
  '1.026359e+01',
  '5.928773e+00',
  '2.069658e+00',
  '0.000000e+00',
  '0.000000e+00',
  '8.755843e-01',
  '0.000000e+00',
  '1.619236e+01',
  '3.635236e+00',
  '2.687283e+01',
  'LWPs/LWP684/C3',
  '3',
  '5',
  '1',
  '1.246656e+00',
  '6.142371e-01',
  '1.611205e+00',
  '1.229054e+00',
  '1.245192e+00',
  '8.084302e-01',
  '1.837196e+00',
  '5.936167e+00',
  '3.648693e+00',
  '1.318086e+00',
  '2.834942e+00',
  '1.410291e+00',
  '3.441351e+00',
  '4.078693e+00',
  '7.486150e-01',
  '5.836648e-01',
  '2.104914e-01',
  '1.649530e+00',
  '6.939538e-01',
  '1.490323e+00',
  '4.491318e-01',
  '4.645096e+00',
  'LWPs/LWP684/C3',
  '3',
  '5',
  '1',
  '1.202354e+00',
  '4.399343e+01',
  '9.377097e+01',
  '0.000000e+00',
  '4.011459e-

In [7]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'NAA',
 'NAAG',
 'PCh',
 'Prop_Glycol',
 'Scyllo',
 'Tau',
 'Threonine',
 'TNAA',
 'TCho',
 'Glx']

In [8]:
df

,LWP,Row,Col,Slice,-CrCH2,Ala,Asp,Cr,GABA,GPC,...,water FWHM (PPM),water freq (Hz),WNR,res water amp,baseline dev,max baseline,min baseline,baseline shape,initial residual,final residual
0,LWPs/LWP684/C3,3,5,1,4.269192e+00,0.000000e+00,0.000000e+00,8.886865e+00,0.000000e+00,1.565578e+00,...,4.879284e-02,-1.953125e+00,2.966184e+03,3.937095e-01,3.166634e-02,6.246466e-02,-5.646642e-02,2.657665e-02,6.856375e+00,6.303560e+00
1,LWPs/LWP736/CS,3,5,1,2.827812e+00,7.298758e+00,0.000000e+00,6.870912e+00,1.878009e+00,3.171505e+00,...,5.325134e-02,-3.417969e+00,2.892848e+03,2.572767e-01,3.012668e-02,7.716364e-02,-4.669947e-02,5.626776e-03,7.226832e+00,6.485706e+00
2,LWPs/LWP739/CS,3,5,1,3.494428e+00,3.723995e+00,0.000000e+00,5.049267e+00,6.274798e+00,7.901727e-01,...,5.357857e-02,-2.075195e+00,2.520334e+03,1.242558e-01,3.812372e-02,8.442014e-02,-8.966241e-02,6.646982e-02,7.255203e+00,6.642999e+00
3,LWPs/LWP742/Sp,3,5,1,1.107552e+00,6.806063e+00,0.000000e+00,3.300521e+00,0.000000e+00,1.536242e+00,...,4.161864e-02,-1.831055e+00,2.252324e+03,1.251323e-01,2.353419e-02,5.542185e-02,-4.143543e-02,3.434753e-03,7.477647e+00,6.948065e+00
4,LWPs/LWP748/MR,3,5,1,5.031230e+00,0.000000e+00,1.472923e+00,8.221587e+00,3.193903e+00,3.066556e+00,...,4.284498e-02,-1.464844e+00,2.178430e+03,8.628431e-02,4.579111e-02,9.932615e-02,-8.752261e-02,-2.281495e-02,7.176760e+00,6.502554e+00


In [9]:
file
# row_num
# pull_from_csv(file, row=row_num )

PosixPath('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP748/MRS/C0295714_WIP_CSI_PB_auto_TR2S_SENSE_6_2_raw_act_both.csv')

In [10]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act_both.csv" in f.name.lower() and "csi" in f.name.lower())]
csv_files


[PosixPath('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP684/C3839474_CSI_PB_auto_TR2S_SENSE_5_2_raw_act_both.csv'),
 PosixPath('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP736/CSI/C7166610_WIP_CSI_PB_auto_TR2S_SENSE_6_2_raw_act_both.csv'),
 PosixPath('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP739/CSI/C4116238_WIP_CSI_PB_auto_TR2S_SENSE_8_2_raw_act_both.csv'),
 PosixPath('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP742/Spec/C0688569_WIP_CSI_PB_auto_TR2S_SENSE_6_2_raw_act_both.csv'),
 PosixPath('/Users/patxi/Sync/MRdata/IoN_Piglet/ForRay_22Nov2022/LWPs/LWP748/MRS/C0295714_WIP_CSI_PB_auto_TR2S_SENSE_6_2_raw_act_both.csv')]